In [ ]:
!pip install --user yfinance
!pip install --user tqdm
!pip install --upgrade --user jupyter ipywidgets

In [ ]:
!pip install --user iprogress
!pip install iprogress
!pip install --upgrade --user jupyter ipywidgets
!pip install --upgrade jupyter ipywidgets

In [ ]:
import sqlite3
import json
import yfinance as yf
from tqdm import tqdm, tqdm_notebook
import time

def initialize_database():
    conn = sqlite3.connect('company_info.db')
    c = conn.cursor()
    c.execute('''
              CREATE TABLE IF NOT EXISTS companies
              (id INTEGER PRIMARY KEY,
              company_sec_cik TEXT,
              company_name TEXT,
              company_ticker TEXT,
              company_exchange TEXT,
              country TEXT,
              industry TEXT,
              sector TEXT,
              longBusinessSummary TEXT)
              ''')
    conn.commit()
    conn.close()

def get_company_info(company_sec_cik, company_name, company_ticker, company_exchange):
    company_info = yf.Ticker(company_ticker).info
    return {
        'company_sec_cik': company_sec_cik,
        'company_name': company_name,
        'company_ticker': company_ticker,
        'company_exchange': company_exchange,
        'country': company_info.get('country', 'N/A'),
        'industry': company_info.get('industry', 'N/A'),
        'sector': company_info.get('sector', 'N/A'),
        'longBusinessSummary': company_info.get('longBusinessSummary', 'N/A')
    }

def insert_company_info(company):
    conn = sqlite3.connect('company_info.db')
    c = conn.cursor()
    c.execute('''
              INSERT INTO companies (company_sec_cik, company_name, company_ticker, company_exchange, country, industry, sector, longBusinessSummary)
              VALUES (?, ?, ?, ?, ?, ?, ?, ?)
              ''', (company['company_sec_cik'], company['company_name'], company['company_ticker'], company['company_exchange'],
                    company['country'], company['industry'], company['sector'], company['longBusinessSummary']))
    conn.commit()
    conn.close()

In [ ]:
with open("company_tickers_exchange.json", "r") as f:
    data = json.load(f)["data"]

In [ ]:
initialize_database()

for company in tqdm_notebook(data):

    company_sec_cik = company[0]
    company_name = company[1]
    company_ticker = company[2]
    company_exchange = company[3]
    try:
        company_info = get_company_info(company_sec_cik, company_name, company_ticker, company_exchange)
        insert_company_info(company_info)
        time.sleep(0.1)
    except:
        print(company_sec_cik, company_name, company_ticker)

